# 🍽️ Restaurant Recommendation Agent using AWS Strands with **Observability and Evaluation**

#### What we will build:

- **Local Vector DB**: A searchable collection of restaurant information that the agent can query.
- **Strands Agent**: An AI assistant that can recommend restaurants based on their preferences.
- **LangFuse**: A tool that lets us "see" how our agent works and makes decisions.
- **RAGAS**: A framework that helps us evaluate how well our agent is performing.

### 📦 Required packages

- **langchain** - helps with splitting documents into chunks
- **langfuse** - observability for the agent
- **ragas** - evaluate agent's performanse
- **chromadb** - storing and searching vector embeddings
- **boto3** - AWS SDK for Python, access AWS services and use AWS Bedrock models
- **sentence-transformers** - using Huggingface library for text embeddings
- **strands-agents** - building AI agents

### 💿 Create vectorstore from Restaurant data

A Vector database stores text as vectors that represent the meaning of the text. This allows us to search for similar meanings, not just extract word matches. 

**For example:** If we search for *"vegetarian food"*, we might also find results about *"plant-based dishes"*, even if those exact words aren't used. 

Create a vector database using data files in the `data` folder. Those are **Markdown** files containing information about different restaurants, their menus and their specialties. 

#### Load enviroment variables

In [18]:
import os
from dotenv import load_dotenv

load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
if not OPENAI_API_KEY:
    print("⚠️ OPENAI_API_KEY environment variable not set")
else:
    print(f"✅ OPENAI_API_KEY environment variable set: {OPENAI_API_KEY[:5]}...{OPENAI_API_KEY[-5:]}")

HF_TOKEN = os.getenv("HF_TOKEN")
if not HF_TOKEN:
    print("⚠️ HF_TOKEN environment variable not set")
else:
    print(f"✅ HF_TOKEN environment variable set: {HF_TOKEN[:5]}...{HF_TOKEN[-5:]}")

GROQ_API_KEY = os.getenv("GROQ_API_KEY")
if not GROQ_API_KEY:
    print("⚠️ GROQ_API_KEY environment variable not set")
else:
    print(f"✅ GROQ_API_KEY environment variable set: {GROQ_API_KEY[:5]}...{GROQ_API_KEY[-5:]}")

✅ OPENAI_API_KEY environment variable set: sk-pr...fzbAA
✅ HF_TOKEN environment variable set: hf_zu...hMlKE
✅ GROQ_API_KEY environment variable set: gsk_1...n45ix


In [6]:
# Import necessary libraries for document loading and processing
import os
from langchain_core.documents import Document
from langchain_community.document_loaders.markdown import UnstructuredMarkdownLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from pathlib import Path
import json


In [7]:
DATA_PATH = './data'
assert os.path.exists(DATA_PATH), f"Data path {DATA_PATH} does not exist"

def load_documents(folder_path):
    """
    Load all markdown documents from a folder.
    
    Args:
        folder_path: Path to folder containing markdown files
        
    Returns:
        List of Document objects
    """
    folder = Path(folder_path)
    
    if not folder.exists():
        raise ValueError(f"Folder not found: {folder_path}")
    
    # Find all markdown files in the folder
    md_files = list(folder.glob("*.md"))
    
    if not md_files:
        print(f"Warning: No markdown files found in {folder_path}")
        return []
    
    print(f"Found {len(md_files)} markdown files in {folder_path}")
    print("=" * 60)
    
    all_documents = []
    
    # Loop through each markdown file
    for md_file in md_files:
        print(f"Loading: {md_file.name}")
        
        # Load the markdown document
        loader = UnstructuredMarkdownLoader(str(md_file))
        documents = loader.load()
        
        # Add source metadata
        for doc in documents:
            doc.metadata['source_file'] = md_file.name
            doc.metadata['source_path'] = str(md_file)
        
        all_documents.extend(documents)
        print(f"  - Loaded {len(documents)} document(s)")
    
    print("=" * 60)
    print(f"📚 Total documents loaded: {len(all_documents)}\n")
    
    return all_documents

all_documents: list[Document] = load_documents(DATA_PATH)

Found 3 markdown files in ./data
Loading: restaurant_bella_luna.md
  - Loaded 1 document(s)
Loading: restaurant_verde_garden.md
  - Loaded 1 document(s)
Loading: restaurant_dragon_palace.md
  - Loaded 1 document(s)
📚 Total documents loaded: 3



In [9]:
def split_documents(documents, chunk_size=100, chunk_overlap=10):
    """
    Split documents into chunks.
    
    Args:
        documents: List of Document objects to split
        chunk_size: Maximum size of each chunk in characters
        chunk_overlap: Number of characters to overlap between chunks
        
    Returns:
        List of document chunks
    """
    if not documents:
        print("Warning: No documents to split")
        return []
    
    print(f"Splitting {len(documents)} documents into chunks...")
    print(f"  - Chunk size: {chunk_size} characters")
    print(f"  - Chunk overlap: {chunk_overlap} characters")
    print("=" * 60)
    
    # Initialize the text splitter
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
        length_function=len,
        separators=["\n\n", "\n", " ", ""]
    )
    
    # Split all documents into chunks
    chunks = text_splitter.split_documents(documents)
    
    print(f"Created {len(chunks)} total chunks")
    print("=" * 60)
    
    return chunks

chunks = split_documents(all_documents)

Splitting 3 documents into chunks...
  - Chunk size: 100 characters
  - Chunk overlap: 10 characters
Created 325 total chunks


### Set up Embeddings

Mathematical representations of text (vectors). Convert all the chunks into vectors, so they can be searched efficiently. 

For simplicity, use `all-MiniLM-L6-v2` embedding model from `HuggingFace`.

In [14]:
from langchain_community.embeddings import HuggingFaceEmbeddings
model_name = "sentence-transformers/all-MiniLM-L6-v2"
embeddings = HuggingFaceEmbeddings(model_name=model_name)

### 📊 Create and persist Chroma DB

In [15]:
from langchain_community.vectorstores import Chroma


# Define the directory where Chroma will be saved
# This allows later reuse of the same Vector store
persist_directory = './chroma_db'

# Create the Chroma vector store from chunks
vectorstore = Chroma.from_documents(
    documents=chunks,
    embedding=embeddings,
    persist_directory=persist_directory
)

print(f"✅ Vector store created successfully at {persist_directory}")


✅ Vector store created successfully at ./chroma_db


### Test the vectorstore

Test vectorstore with simple query to make sure, it works as expected. 

In [16]:
query = "What are the best salads?"
docs = vectorstore.similarity_search(query, k=3)

print(f"❓ Query: {query}")
print("📚 Results:")
for doc in docs:
    print(f"  - {doc.metadata['source_file']}: {doc.page_content[:100]}...")

❓ Query: What are the best salads?
📚 Results:
  - restaurant_verde_garden.md: Soups & Salads...
  - restaurant_bella_luna.md: Insalate (Salads)...
  - restaurant_verde_garden.md: Caesar Salad - $12 Romaine lettuce, house-made Caesar dressing (cashew-based), crispy chickpeas,...


### ✅ Set up LangFuse for observability 

##### What is LangFuse?
LangFuse is like a dashboard for AI agents. It helps you see what's happening inside your agent - what questions are asked, how it's thinking about them, and what answers are generated. This is incredibly useful for debbuging AI agents.

Let's configure **LangFuse** for observabillity. We need to create a **LangFuse** Account and get an *API_KEY*.

From **LangFuse Cloud** we should get `LANGFUSE_SECRET_KEY`, `LANGFUSE_PUBLIC_KEY` and `LANGFUSE_BASE_URL`. Currently they are configured in `.env` file.

In [19]:
LANGFUSE_SECRET_KEY = os.getenv("LANGFUSE_SECRET_KEY")
if not LANGFUSE_SECRET_KEY:
    print("⚠️ LANGFUSE_SECRET_KEY environment variable not set")
else:
    print(f"✅ LANGFUSE_SECRET_KEY environment variable set: {LANGFUSE_SECRET_KEY[:5]}...{LANGFUSE_SECRET_KEY[-5:]}")

LANGFUSE_PUBLIC_KEY = os.getenv("LANGFUSE_PUBLIC_KEY")
if not LANGFUSE_PUBLIC_KEY:
    print("⚠️ LANGFUSE_PUBLIC_KEY environment variable not set")
else:
    print(f"✅ LANGFUSE_PUBLIC_KEY environment variable set: {LANGFUSE_PUBLIC_KEY[:5]}...{LANGFUSE_PUBLIC_KEY[-5:]}")

LANGFUSE_BASE_URL = os.getenv("LANGFUSE_BASE_URL")
if not LANGFUSE_BASE_URL:
    print("⚠️ LANGFUSE_BASE_URL environment variable not set")
else:
    print(f"✅ LANGFUSE_BASE_URL environment variable set: {LANGFUSE_BASE_URL}")

✅ LANGFUSE_SECRET_KEY environment variable set: sk-lf...42e91
✅ LANGFUSE_PUBLIC_KEY environment variable set: pk-lf...22962
✅ LANGFUSE_BASE_URL environment variable set: https://cloud.langfuse.com


In [20]:
# Configure LangFuse for observability
from langfuse import Langfuse

langfuse_client = Langfuse(
    public_key=LANGFUSE_PUBLIC_KEY,
    secret_key=LANGFUSE_SECRET_KEY,
    host=LANGFUSE_BASE_URL
)

In [21]:
# Make sure we can access the LangFuse client
from langfuse import get_client

# Access the client
langfuse_client = get_client(public_key=LANGFUSE_PUBLIC_KEY)
print("✅ LangFuse client created successfully")

# Flush all pending observations
# This ensures all data is sent to Langfuse
langfuse_client.flush()

✅ LangFuse client created successfully


### 🤖 Create a Restaurant Recommendation Agent

Create `Strands` agent that uses the Chroma vector database to provide restaurants recommendations. The agent will:

1. Receive question from users about restaurants
2. Search our vector database for relevant information
3. Generate helpful responses based on search results


In [22]:
# Import necessary libraries for agent
from strands.agent import Agent # The main agent class
from strands.models.openai import OpenAIModel # OpenAI model
from strands.tools import tool # Tool decorator

In [23]:
# Set up OpenAI model
openai_model = OpenAIModel(
    # Set up authentication using API key from environment variables 
    client_args={
        'api_key': OPENAI_API_KEY
    },
    # Model config
    model_id="gpt-4o", # Specify which OpenAI model version to use
    params={
        'temperature': 0.0, # Set temperature to 0 for deterministic responses
        'max_tokens': 2048 # Set max response length 
    },
)

In [24]:
from strands.hooks import HookProvider, HookRegistry
from strands.hooks.events import AfterToolCallEvent, BeforeToolCallEvent

class ToolLoggingHook(HookProvider):
    """
    A simple hook to log tool names and arguments before they are executed.
    """
    
    def register_hooks(self, registry: HookRegistry) -> None:
        # Register the callback for the 'BeforeToolCallEvent'
        registry.add_callback(BeforeToolCallEvent, self.log_tool_usage)
        registry.add_callback(AfterToolCallEvent, self.log_tool_result)

    def log_tool_usage(self, event: BeforeToolCallEvent) -> None:
        print("\n" + "="*50)
        print(f"🚀  **INVOKING TOOL:** {event.tool_use['name']}")
        print("-" * 50)
        print("📥  **Arguments:**")
        print(self._pretty_format(event.tool_use["input"]))
        print("="*50 + "\n")

    def log_tool_result(self, event: AfterToolCallEvent) -> None:
        print("\n" + "="*50)
        print(f"✅  **TOOL COMPLETED:** {event.tool_use['name']}")
        print("-" * 50)
        print("📤  **Result:**")
        # Handle cases where result might be a dict or a plain string
        try:
            # If the result is a JSON string/dict, format it
            content = event.result['content']
            if isinstance(content, (dict, list)):
                 print(self._pretty_format(content))
            else:
                 print(f"    {content}")
        except Exception:
            # Fallback for complex objects
            print(f"    {event.result}")
            
        print("="*50 + "\n")

    def _pretty_format(self, data) -> str:
        """Helper to format dicts/lists as indented JSON string."""
        try:
            return json.dumps(data, indent=4, ensure_ascii=False)
        except TypeError:
            return str(data)

In [25]:
# Create retrieval tool using Chroma DB
# This tool will allow the agent to search for relevant information in the vector database
@tool
def restaurant_retrieval(query: str) -> str:
    """Search for restaurant information based on cuisine, dietary preferences, location or other criteria
    
    Args:
        query: Search query

    Returns:
        str: Information about restaurants matching the query
    """
    # Load persisted Chroma DB
    vectorstore = Chroma(
        persist_directory=persist_directory,
        embedding_function=embeddings
    )

    # Perform similarity search
    docs = vectorstore.similarity_search(query, k=3)

    # Format search results as a string
    results = "\n".join([f"{doc.metadata['source_file']}: {doc.page_content}" for doc in docs])

    return results if results else "No relevant information found"


In [27]:
import uuid # For generating unique IDs

# Create Restaurant Recommendation Agent
agent = Agent(
    model=openai_model,
    name='Restaurant Recommendation Agent',
    tools=[restaurant_retrieval], # Give the agent access to retrieval tool
    hooks=[ToolLoggingHook()],
    system_prompt="""You are a helpful restaurant recommendation assistant.
    Use the restaurant_retrieval tool to find information about restaurants in your database.
    Provide detailed recommendations based on user queries and search results.

    If asked about restaurants that are NOT in the database, politely explain that you can only provide information about restaurants in your database.

    Always be friendly, helpful and concise in your responses.
    """,
    callback_handler=None,
    record_direct_tool_call=True,
    trace_attributes={
        'session.id': str(uuid.uuid4()), # Generate Unique session ID
        'user.id': 'dimodrangov@gmail.com', # Example user ID
        'langfuse.tags': [
            'Agent-SDK Example',
            'Restaurant-Recommendation',
            'Strands-Project-Demo',
            'Observability-Tutorial'
        ],
    }
)

### 🧪 Test the agent with traces

Test the agent with a simple query and see how it performs. The agent will use the retrieval tool to find relevant information and generate a response 

In [28]:
# Test the agent with a simple query
query = "I'm looking for a restaurant with goot vegetarian options. Any recommendations"
response = agent(query)
print(f'Agent response:\n{response}')


🚀  **INVOKING TOOL:** restaurant_retrieval
--------------------------------------------------
📥  **Arguments:**
{
    "query": "vegetarian options"
}



/var/folders/g9/x11cjlw56ln82_76czqsvqvr0000gn/T/ipykernel_97190/986610298.py:14: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the `langchain-chroma package and should be used instead. To use it run `pip install -U `langchain-chroma` and import as `from `langchain_chroma import Chroma``.
  vectorstore = Chroma(



✅  **TOOL COMPLETED:** restaurant_retrieval
--------------------------------------------------
📤  **Result:**
[
    {
        "text": "restaurant_verde_garden.md: Specialties\n\n100% Plant-Based: Everything on our menu is vegan - no animal products whatsoever\nrestaurant_bella_luna.md: We offer gluten-free pasta options and can accommodate vegetarian requests. Please inform your\nrestaurant_dragon_palace.md: with vegetables and choice of protein"
    }
]

Agent response:
Here are some restaurants with good vegetarian options:

1. **Verde Garden**: This restaurant is 100% plant-based, meaning everything on their menu is vegan and free from animal products. It's a great choice if you're looking for a fully vegetarian dining experience.

2. **Bella Luna**: They offer gluten-free pasta options and can accommodate vegetarian requests. Just inform your server, and they'll be happy to assist.

3. **Dragon Palace**: They offer dishes with vegetables and a choice of protein, which can be custo

### 📈 Review the traces

After running the agent review the traces in Langfuse:

1. Go to the tracing menu in **LangFuse** project.
2. Select the trace.
3. Examine how the agent processes the request, what tools it used, and what response it generated.

This gives visibility info how the agent is working and helps identifying any issues or areas for improvement.

### 🤔 Set up **RAGAS** for evaluation 

Using **RAGAS** for evaluating the quality of agent's responses. 
**RAGAS** (Retrieval Augmented Generation Assesment) is a framework for evaluating **RAG** systems. 

##### What is RAGAS?
RAGAS helps us measure how well our agent is performing by looking at different aspects of its responses:
- Is the information relevant?
- Is it relevant to the user's question?
- Is it using the right tools?
- Is it communicating in a friendly way?



In [ ]:
# Import RAGAS library
from openai import OpenAI
from ragas.llms import llm_factory

# Setup the evaluation LLM
evaluation_llm = llm_factory('gpt-4o-mini', client=OpenAI(api_key=OPENAI_API_KEY))

/var/folders/g9/x11cjlw56ln82_76czqsvqvr0000gn/T/ipykernel_97190/1471895289.py:5: DeprecationWarning: LangchainLLMWrapper is deprecated and will be removed in a future version. Use llm_factory instead: from openai import OpenAI; from ragas.llms import llm_factory; llm = llm_factory('gpt-4o-mini', client=OpenAI(api_key='...'))
  evaluation_llm = LangchainLLMWrapper(openai_model)


### Define RAGAS metrics

Define several metrics to evaluate different aspects of our agent's performance

In [31]:
from ragas.metrics._aspect_critic import AspectCritic

# Metric to check if the agent fulfills all user requests
request_completeness = AspectCritic(
    name="request_completeness",
    definition="Return 1 if the agent completely fulfills all the user requests with no ommisions, otherwise return 0",
    llm=evaluation_llm
)

# Metric to assess if the AI's communication alligns with the desired brand voice
brand_tone = AspectCritic(
    name="brand_tone",
    definition="Return 1 if the AI's communication is Friendly, approachable, helpful, clear and concise, otherwise return 0",
    llm=evaluation_llm
)

# Tool usage effectiveness
tool_effectiveness = AspectCritic(
    name="tool_effectiveness",
    definition="""Return 1 if the agent appropriately used available tools to fulfill the user request
    (such as using retrieve for menu questions and current_time for time questions).
    Return 0 if the agent failed to use appropriate tools to fulfill the user request""",
    llm=evaluation_llm
)

# Tool selection appropriateness
tool_selection = AspectCritic(
    name="tool_selection",
    definition="""Return 1 if the agent appropriately selected the right tools to fulfill the user request
    (such as using retrieve for menu questions and current_time for time questions).
    Return 0 if the agent failed to select the right tools to fulfill the user request""",
    llm=evaluation_llm
)

Let's also define metric to evaluate RAG (Retrievement Augmented Generation) of our agent. 